<a href="https://colab.research.google.com/github/Jean2Pierre/BV_Case_JPB/blob/main/BV_Case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.0 Before going any further, let's get the permissions and data from Google Drive ⬆ 

---



In [ ]:
# 1.1 Get Google Colab permission to import data from Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

# 1.2 Importing pandas to help us deal with data and dataframes
import pandas as pd

# 1.3 Importing SQL to help us handle data and specially sqldf to to give us a hand transforming sql queries into dataframes
!pip install -U pandasql
import pandasql as ps
from pandasql import sqldf


2.0 Now... it's time to bring the data in...

---



In [2]:
# 2.1 Time to bring the data in...

df_country = pd.read_csv('/content/gdrive/MyDrive/BV_database/Bases/Country.csv', sep=';')
df_region = pd.read_csv('/content/gdrive/MyDrive/BV_database/Bases/Region.csv', sep=';')
df_type = pd.read_csv('/content/gdrive/MyDrive/BV_database/Bases/Type.csv', sep=';')
df_exportacao = pd.read_csv('/content/gdrive/MyDrive/BV_database/Bases/Exportacao.csv', sep=';')


3.0 Data cleanning, formatting and organizing 

---



In [ ]:
# 3.1 Checking for Null Values on df_country dataframe

df_country.isna().sum()

In [ ]:
# 3.2 Checking for Null Values on df_region dataframe

df_region.isna().sum()

In [ ]:
# 3.3 Checking for Null Values on df_type dataframe

df_type.isna().sum()

In [ ]:
# 3.4 Checking for Null Values on df_exportacao dataframe

df_exportacao.isna().sum()

In [ ]:
# 3.5 Checking data type - df_country dataframe

df_country.info()

In [ ]:
# 3.6 Checking data type - df_region dataframe

df_region.info()

In [ ]:
# 3.7 Checking data tye - df_type dataframe

df_type.info()

In [ ]:
# 3.8 Checking data tye - df_exportacao dataframe

df_exportacao.info()

In [ ]:
# 3.9 It is already known that hackers have broken into the dataset and messed up the "Exportacao" table 
  # After checking all columns, Date format were found to be wrong on "Order Date" and "Ship Date" columns
  
sql_query1 = sqldf("SELECT distinct [Order Date]  FROM df_exportacao ")
sql_query1.head()


In [ ]:
# 3.10 As you can see there are Order ID duplicates. Basically, hackers have duplicated some orders and messed up dates

sql_query2 = "SELECT count ([Order ID]) FROM df_exportacao GROUP by [Order ID] "
print(ps.sqldf(sql_query2, locals()))

In [13]:
# 3.11 Putting the Order Date column into the righ format. With this, we'll fix the fris problem.
df_exportacao['Order_date_check'] = pd.to_datetime(df_exportacao['Order Date'], format='%m/%d/%Y', errors='coerce').fillna(pd.to_datetime(df_exportacao['Order Date'], format='%m/%d/%Y', errors='coerce')).astype(str)
df_exportacao['Ship_Date_check'] = pd.to_datetime(df_exportacao['Ship Date'], format='%m/%d/%Y', errors='coerce').fillna(pd.to_datetime(df_exportacao['Ship Date'], format='%m/%d/%Y', errors='coerce')).astype(str)


In [14]:
# 3.12 Getting all the main datadrame treated - this is the main SQL dataframe we're sending to Goole Sheets/Google Data Studio

sql_query_0 = sqldf("""
                        SELECT
                              *
                        FROM df_exportacao e
                        WHERE Order_date_check = (SELECT MIN(Order_date_check) FROM df_exportacao ee WHERE e.[Order ID]= ee.[Order ID])                              
                   """)


4.0 SQL QUERIES AND PROBLEM SOLVING

---



 4.1 "Traga um relatório completo da divergência existente"

In [39]:
# 4.1.1 Counting how many Order IDs were duplicated
sql_query3 = sqldf( """            
            SELECT 
                  count(distinct [Order ID]) As Contagem_unica_IDs,
                  count([Order ID]) As Contagem_total_IDs,
                  (count([Order ID]) -count(distinct [Order ID])) As Contagem_IDs_duplicados
            FROM  df_exportacao         
             """)
sql_query3.head()

# RESUMO: 429 IDs foram duplicados resultando no aumento total de pedidos. O número correto seriam de 10.000 pedidos e não 10.429

,Contagem_unica_IDs,Contagem_total_IDs,Contagem_IDs_duplicados
0,10000,10429,429


In [38]:
# 4.1.2 Duplicated gathered by Country, Region, Item_Type, Sales Channel
sql_query4 = sqldf( """
             WITH duplication As (
                SELECT 
                      [Order ID],
                      ID_type,
                      ID_Region,
                      ID_Country,
                      [Sales Channel],
                      Receita, 
                      COUNT([Order ID]) As Is_duplicated,
                      [Units Sold]
                FROM df_exportacao
                GROUP by 1)

             SELECT 
                  
                  c.Country,
                  r.Region,
                  t.Item_Type,
                  d.[Sales Channel],
                  SUM(Receita) As volume_receita_errada,
                  SUM([Units Sold]) As Volume_unidades_erradas,
                  COUNT([Order ID]) As Volume_divergências                              
             FROM duplication d
             LEFT join df_type t ON d.ID_type = t.ID_Type
             LEFT join df_region r ON d.ID_Region = r.ID_Region 
             LEFT join df_country c ON d.ID_Country = c.ID_Country                
             WHERE Is_duplicated = 2 
             ORDER by 1, 2, 3, 4                       
             """)

sql_query4.head()

# RESUMO: Os pedidos duplicados encontram-se todos concentrados em um país: Grenada. Além disso, com essa invasão, foram adicionadas
# erroneamente na tabela cerca de 2162698	unidades de cereal a mais. Caso esta invasão hacker não tivesse sido detectada, a receita
# no caixa da empresa estaria inflada em  160828520.0 US$

,Country,Region,Item_Type,Sales Channel,volume_receita_errada,Volume_unidades_erradas,Volume_divergências
0,Grenada,Central America and the Caribbean,Cereal,Online,160828520.0,2162698,429


In [ ]:
# 4.1.3 There's also errors on both date columns

sql_query_date_1 = sqldf("SELECT distinct [Order Date]  FROM df_exportacao ")
sql_query_date_1.head()

# RESUMO: É possível verifica que há troca de mês e dia na base de dados, coluna Order Date. 

In [ ]:
# 4.1.4 There's also errors on both date columns

sql_query_date_2 = sqldf("SELECT distinct [Ship Date]  FROM df_exportacao ")
sql_query_date_2.head()

# RESUMO: É possível verifica que há troca de mês e dia na base de dados, coluna Order Date.

In [ ]:
# 4.1.5 So, this is de real dataset, without the duplicates
sql_query5 = sqldf("""
              WITH cte_a As(
                SELECT 
                      [Order ID],
                      Order_date_check
                FROM df_exportacao
              )
              
              SELECT 
                    [ORDER ID],
                    Order_date_check
              FROM df_exportacao e
              WHERE Order_date_check = (SELECT MIN(Order_date_check) FROM df_exportacao ee WHERE e.[Order ID]= ee.[Order ID])

             """)

sql_query5.head()


# RESUMO: esse seria a base de dados correta, sem os 429 registros duplicados.

4.2 Perguntas

In [ ]:
# 1- Analisando ano a ano (2010,2011...), qual foi o item que mais teve ordem de pedidos emitidos (Order_ID) no mês de abril ao longo dos anos?
  # Mostrando apenas o que foi pedido na pergunta.
query_questao_um_short = sqldf("""
                            WITH CTE_1 As (                           
                              SELECT
                                    strftime('%Y', Order_date_check) As Ano_pedido,                                    
                                    t.Item_Type As type,
                                    count([Order ID]) As volume_pedidos
                              FROM df_exportacao e
                              LEFT join df_type t ON e.ID_Type = t.ID_Type
                              WHERE strftime('%m', Order_date_check) = '04'
                                    and Order_date_check = (SELECT MIN(Order_date_check) FROM df_exportacao ee WHERE e.[Order ID]= ee.[Order ID])
                              GROUP by 1, 2
                              ORDER by 1 desc)

                              SELECT 
                                    type,
                                    SUM(volume_pedidos)                                    
                              FROM CTE_1                               
                              GROUP by 1 
                              ORDER by 2 desc
                              LIMIT 1                
                        """)

#query_questao_um.pivot_table(index='type', columns='Ano_pedido', values='volume_pedidos', aggfunc='sum', fill_value=0, margins=True)

query_questao_um_short.head()


In [ ]:
# 1- Analisando ano a ano (2010,2011...), qual foi o item que mais teve ordem de pedidos emitidos (Order_ID) no mês de abril ao longo dos anos?
  # Mostrando agora o que foi pedido acima e colocando também as demais categorias de item para ficar legal a comparação.
query_questao_um_open = sqldf("""                                                      
                              SELECT
                                    strftime('%Y', Order_date_check) As Ano_pedido,                                    
                                    t.Item_Type As type,
                                    count([Order ID]) As volume_pedidos
                              FROM df_exportacao e
                              LEFT join df_type t ON e.ID_Type = t.ID_Type
                              WHERE strftime('%m', Order_date_check) = '04'
                                    and Order_date_check = (SELECT MIN(Order_date_check) FROM df_exportacao ee WHERE e.[Order ID]= ee.[Order ID])
                              GROUP by 1, 2
                              ORDER by 1 desc
                                         
                        """)

query_questao_um_open.pivot_table(index='type', columns='Ano_pedido', values='volume_pedidos', aggfunc='sum', fill_value=0, margins=True)




In [ ]:
# 2- Qual foi o país com o maior faturamento da Ásia no ano de 2017.

query_questao_dois = sqldf("""
                              SELECT
                                    c.Country,                                                                        
                                    SUM(faturamento) As faturamento            
                              FROM df_exportacao e 
                              LEFT join df_country c ON e.ID_Country = c.ID_Country
                              LEFT join df_region r ON e.ID_Region = r.ID_Region
                              WHERE r.Region = 'Asia'
                                    and strftime('%Y', Order_date_check) = '2017'
                                    and Order_date_check = (SELECT MIN(Order_date_check) FROM df_exportacao ee WHERE e.[Order ID]= ee.[Order ID])
                              GROUP by 1
                              ORDER by 2 desc
                              LIMIT 1
                           """)

query_questao_dois.pivot_table(index='Country', values='faturamento', fill_value=0).sort_values( by='faturamento', ascending=False)

In [ ]:
# 3- Qual foi o país com o maior custo da Ásia no ano de 2014.

query_questao_tres = sqldf ("""
                             SELECT
                                    c.Country,                                                                        
                                    SUM(Custos) As Custos            
                              FROM df_exportacao e 
                              LEFT join df_country c ON e.ID_Country = c.ID_Country
                              LEFT join df_region r ON e.ID_Region = r.ID_Region
                              WHERE r.Region = 'Asia'
                                    and strftime('%Y', Order_date_check) = '2014'
                                    and Order_date_check = (SELECT MIN(Order_date_check) FROM df_exportacao ee WHERE e.[Order ID]= ee.[Order ID])
                              GROUP by 1
                              ORDER by 2 desc
                              LIMIT 1
                            """)

query_questao_tres.pivot_table(index='Country', values='Custos', fill_value=0).sort_values( by='Custos', ascending=False)

In [ ]:
# 4- Qual foi o país com a maior Receita da Ásia no ano de 2012.
query_questao_quatro = sqldf ("""
                             SELECT
                                    c.Country,                                                                        
                                    SUM(Receita) As Receita            
                              FROM df_exportacao e 
                              LEFT join df_country c ON e.ID_Country = c.ID_Country
                              LEFT join df_region r ON e.ID_Region = r.ID_Region
                              WHERE r.Region = 'Asia'
                                    and strftime('%Y', Order_date_check) = '2012'
                                    and Order_date_check = (SELECT MIN(Order_date_check) FROM df_exportacao ee WHERE e.[Order ID]= ee.[Order ID])
                              GROUP by 1
                              ORDER by 2 desc
                              LIMIT 1
                            """)

query_questao_quatro.pivot_table(index='Country', values='Receita', fill_value=0).sort_values( by='Receita', ascending=False)


In [60]:
# 5- A área de produtos precisa de algumas informações vitais para a manutenção do portifólio. Qual o tipo de produto mais vendido (Units_Sold) na somatória de todos os anos?

query_questao_cinco = sqldf("""
                              SELECT                                                                     
                                    t.Item_Type As type,
                                    SUM([Units Sold]) As volume_pedidos_vendidos
                              FROM df_exportacao e
                              LEFT join df_type t ON e.ID_Type = t.ID_Type
                              WHERE Order_date_check = (SELECT MIN(Order_date_check) FROM df_exportacao ee WHERE e.[Order ID]= ee.[Order ID])
                              GROUP by 1
                              LIMIT 1
                        """)

query_questao_cinco.pivot_table(index='type', fill_value=0).sort_values( by='volume_pedidos_vendidos', ascending=False)


,volume_pedidos_vendidos
type,
Baby Food,4146333


5.0 SENDING DATA TO GOOGLE SHEETS 

In [23]:
# 4.1 Getting all permissions to send data to Google Sheets
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
wb = gc.open_by_key('190DYU0m_SFqv-8SQ0mTIXImsMS2msHQDjQXeoGhfcTk')

In [ ]:
!pip install --upgrade gspread

In [ ]:
# 4.2 Populating the REGION worksheet on Google Sheets

ws = wb.worksheet('Country')
cabecalho_country = ['Country', 'ID_Country'] 
todos_registros_country = [cabecalho_country] + df_country.to_numpy().tolist()
ws.update(None, todos_registros_country)


In [ ]:
# 4.3 Populating the COUNTRY worksheet on Google Sheets

ws = wb.worksheet('Region')
cabecalho_region = ['Region', '	ID_Region'] 
todos_registros_region = [cabecalho_region] + df_region.to_numpy().tolist()
ws.update(None, todos_registros_region)


In [ ]:
# 4.4 Populating the TYPE worksheet on Google Sheets

ws = wb.worksheet('Type')
cabecalho_type = ['ID_Type', 'Item_Type']   
todos_registros_type = [cabecalho_type] + df_type.to_numpy().tolist()
ws.update(None, todos_registros_type)

In [ ]:
# 4.5 Populating the EXPORTACAO worksheet on Google Sheets

ws = wb.worksheet('Exportacao')
cabecalho_exportacao = ['ID_Region', 'ID_Country',	'ID_Type', 'Sales Channel', 'Order Priority', 'Order Date', 'Order ID',	
'Ship Date', 'Units Sold', 'Unit Price', 'Unit Cost',	'Faturamento', 'Custos',	'Receita', 'Order_date_check', 'Ship_Date_check']
todos_registros_exportacao = [cabecalho_exportacao] + sql_query_0.to_numpy().tolist()
ws.update(None, todos_registros_exportacao)